In [1]:
using Gen
using Serialization
using Logging
using BenchmarkTools

In [2]:
using GenArrow

In [3]:
function write_to_file(fname, input)
    seekstart(input)
    data = read(input, String)
    open(fname, "w") do io
        write(io, data)
    end
end
function read_from_file(fname)
    io = IOBuffer()
    open(fname, "r") do file
        data = read(file, String)
        write(io, data)
    end
    seekstart(io)
    io
end

read_from_file (generic function with 1 method)

### Leaf Nodes

In [4]:
@gen function submodel(w)
    a ~ bernoulli(0.5)
    b ~ bernoulli(0.5)
end

@gen function model(n)
    z ~ bernoulli(0.5)
    # for k=1:n
    #     @trace(bernoulli(0.5), k)
    # end
    q ~ submodel("what the")
end

DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[Any], false, Union{Nothing, Some{Any}}[nothing], var"##model#313", Bool[0], false)

In [5]:
debugIO = open("write.txt", "w+")
logger = ConsoleLogger(debugIO, Debug)
global_logger(logger)
tr_old, w_old = generate(model, (10,))
display(get_choices(tr_old))
display(get_score(tr_old))
io = IOBuffer()
GenArrow.serialize(io, tr_old)
close(debugIO)
write_to_file("data.trace", io)

│
├── :z : true
│
└── :q
    │
    ├── :a : true
    │
    └── :b : false


-2.0794415416798357

538

In [7]:
debugIO = open("read.txt", "w+")
logger = ConsoleLogger(debugIO, Debug)
global_logger(logger)

io = read_from_file("data.trace")
recovered_tr = try
    recovered_tr = GenArrow._deserialize(model, io, true)
    close(debugIO)
    recovered_tr
catch
    close(debugIO)
    rethrow()
end
# display(get_score(recovered_tr))
# display(get_choices(recovered_tr))
# display(get_args(recovered_tr))

LazyTrace(Ptr: -1, Length: -1
[L]
 ┣━━ z ━ Gen.ChoiceOrCallRecord{Bool}(true, -0.6931471805599453, NaN, true)
 ┣━━ q ━ (record_ptr = 252, record_size = 286, is_trace = true)
[I]
, IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=538, maxsize=Inf, ptr=226, mark=-1), false, -2.0794415416798357, 0.0, (10,), false)